In [28]:
import os
import numpy as np
import pandas as pd
import SimpleITK as sitk

joint = "hip" # hip, knee, ankle
ds = "val" # val, test

image_dir = f"data/images/4_pre-proc3/{joint}"
landmarks_array = np.load(f"data/arrays/y_{ds}_{joint}.npy")
landmarks_pred_array = np.load(f"data/arrays/y_pred_{ds}_{joint}.npy")

In [38]:
def create_mask(image_ref):
    """
    Create a mask with the same properties as the input image.

    Parameters:
    - image_ref (sitk.Image): The reference image to create the mask from.

    Returns:
    - mask (sitk.Image): The created mask with the same properties as the input image.
    """
    
    mask = sitk.Image(image_ref.GetSize(), sitk.sitkUInt8)
    mask.SetSpacing(image_ref.GetSpacing())
    mask.SetOrigin(image_ref.GetOrigin())
    mask.SetDirection(image_ref.GetDirection())
    
    return mask

def apply_landmarks(image, landmarks_pred_array, landmarks_array, image_index):
    """
    Set voxel value to 1 at each landmark using a NumPy array for landmark coordinates.

    Parameters:
    - image (sitk.Image): The input image data.
    - landmarks_pred_array (numpy.ndarray): The predicted landmarks array.
    - landmarks_array (numpy.ndarray): The annotated landmarks array.
    - image_index (int): The index of the image to process.

    Returns:
    - image (sitk.Image): The image with landmarks applied.
    """

    print("------Applying landmarks------")

    # Calculate the number of landmarks based on the array shape
    n_landmarks = landmarks_pred_array.shape[1] // 3

    # Extract the landmarks for the specified image
    image_landmarks_pred = landmarks_pred_array[image_index, :]
    image_landmarks = landmarks_array[image_index, :]
    
    for i in range(n_landmarks):
        # Each landmark has 3 consecutive values (x, y, z)
        start_idx = i * 3
        x, y, z = image_landmarks_pred[start_idx:start_idx + 3]

        # Round the coordinates to nearest integer
        x, y, z = round(x), round(y), round(z)
        landmark_coords = (x, y, z)

        print(f"Predicted landmark {i+1}: {landmark_coords}")

        try:
            # Attempt to set voxel value to 1
            image.SetPixel(landmark_coords, 1)
        except Exception as e:
            # Handle any error by skipping this landmark and printing a message
            print(f"Skipped applying landmark at coordinates {landmark_coords} due to error: {e}")
            
        
        x, y, z = image_landmarks[start_idx:start_idx + 3]
        x, y, z = round(x), round(y), round(z)
        landmark_coords = (x, y, z)

        print(f"Annotated landmark {i+1}: {landmark_coords}")

    return image

def binary_dilate(image):
    """
    Apply a Binary Dilation filter to make intensity 1 voxels become spheres.

    Parameters:
    - image (sitk.Image): The input image to be dilated.

    Returns:
    - dilated_image (sitk.Image): The dilated image.
    """

    # Create a Binary Dilation filter
    dilate_filter = sitk.BinaryDilateImageFilter()
    dilate_filter.SetKernelRadius(3)
    dilate_filter.SetKernelType(sitk.sitkBall)
    dilate_filter.SetForegroundValue(1)

    # Apply the dilation filter to the image
    dilated_image = dilate_filter.Execute(image)

    return dilated_image

In [39]:
# Loop through all images in the directory
image_names = sorted(os.listdir(image_dir))
for image_index, image_name in enumerate(image_names):
    if image_name.endswith('.nii.gz'):
        
        print(f"Image {image_name}, index {image_index}:")
        print("--------------------------------")

        # Load image
        image = sitk.ReadImage(os.path.join(image_dir, image_name))

        # Create mask
        mask = create_mask(image)

        # Apply landmarks prediction to mask
        mask_annot = apply_landmarks(mask, landmarks_pred_array, landmarks_array, image_index)
        
        # Apply binary dilation to visualise landmarks 
        mask_dilated = binary_dilate(mask_annot)
        
        # Save image
        sitk.WriteImage(mask_dilated, os.path.join(image_dir, "mask_pred", image_name[:-7] + f"_mask_pred_{joint}.nii.gz"))
        
        print("")

Image lleg_001_20100125b_hip.nii.gz, index 1:
--------------------------------
------Applying landmarks------
Predicted landmark 1: (107, 124, 153)
Annotated landmark 1: (108, 128, 155)
Predicted landmark 2: (129, 132, 82)
Annotated landmark 2: (128, 126, 76)
Predicted landmark 3: (159, 139, 158)
Annotated landmark 3: (149, 132, 153)

Image lleg_003_20100305b_hip.nii.gz, index 2:
--------------------------------
------Applying landmarks------
Predicted landmark 1: (98, 114, 141)
Annotated landmark 1: (109, 111, 161)
Predicted landmark 2: (119, 122, 74)
Annotated landmark 2: (117, 123, 73)
Predicted landmark 3: (145, 128, 146)
Annotated landmark 3: (158, 150, 150)

Image lleg_004_20100315a_hip.nii.gz, index 3:
--------------------------------
------Applying landmarks------
Predicted landmark 1: (104, 121, 149)
Annotated landmark 1: (112, 113, 155)
Predicted landmark 2: (125, 129, 79)
Annotated landmark 2: (125, 125, 79)
Predicted landmark 3: (154, 135, 154)
Annotated landmark 3: (149, 1

KeyboardInterrupt: 